In [1]:
# game params
total_rounds = 20 # so 40 total turns/generations
total_games = 10
sv_game_history = {}

In [ ]:
def format_prompt(player_id,game_history):
    system_prompt = "Imagine that you and your criminal partner have been placed under arrest and are being interrogated in separate rooms. Both you and your partner have the option to either 'Silence' or 'Confess'. If both of you are silent, you both receive a prison sentence of 1 year. If one of you confesses and the other stays silent, the partner that confesses will receive a prison sentence of 0 years while the one of the stayed silent will receive a sentence of 5 years. If both of you confess, you will both receive sentences of 3 years. Your goal is to minimize your prison sentence. Respond in one word, either 'Silence' or 'Confess'."
    user_prompt = "You are Player " + player_id + ", and this is the history of actions so far: " + game_history + ". Will you choose 'Silence' or 'Confess' in the next round? Please answer in 1 word."

    return system_prompt,user_prompt

def regenerate(result):
    if len(result) >= (len('silence') + len('confess')):
        return True
    # if neither strategy is in the response
    elif (('silence' not in result.lower()) and ('confess' not in result.lower())):
        return True
    # if both strats are in response (sometimes model discusses both)
    elif (('silence' in result.lower()) and ('confess' in result.lower())):
        return True
    # if response is empty
    elif len(result.replace(' ','')) == 0:
        return True
    return False

### options: ####
# 'Confess', 'Silence', 'TitForTat', 'GrimTrigger'
# 'MABandit', 'CBandit'
# 'meta-llama/Llama-2-7b-hf', 'meta-llama/Llama-2-13b-hf', 'meta-llama/Llama-2-70b-hf'
# 'tiiuae/falcon-7b-instruct', 'tiiuae/falcon-40b-instruct'
###

In [2]:
# initialize models
from utils import *
model1_name = "meta-llama/Llama-2-7b-hf"
model1_type = 'hf'
model2_name = "meta-llama/Llama-2-7b-hf"
model2_type = 'hf'
# if models same, don't load twice
model1 = Strategy(model1_name,model1_type)
if model1_name == model2_name:
    model2 = model1
else:
    model2 = Strategy(model2_name,model2_type)

/home/gunala/miniconda3/envs/DialGenEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.16s/it]


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('LLM_IPD')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

# for testing
total_games = 1
total_rounds = 2

for game_id in range(total_games):
    game_history = ""
    sv_game_history[game_id] = {}
    logger.info("Game: " + str(game_id))
    for round_id in range(total_rounds):
        logger.info("Round: " + str(round_id))
        logger.info("generating model 1")
        model1_gen = model1.next_action()
        logger.info("generating model 2")
        model2_gen = model2.next_action()
        # save hist
        model1.add_to_hist(round_id,model1_gen,model2_gen)
        model2.add_to_hist(round_id,model2_gen,model1_gen)

        print('model1 gen: ',model1_gen)
        print('model2 gen: ',model2_gen)

        
        # game_history += "Round " + str(round_id) + ": " + "Player 1 played " + model1_gen + \
        #                  ", Player 2 played " + model2_gen + ". "
        # # save game hist
        # sv_game_history[game_id][round_id] = {'p1':model1_gen,'p2':model2_gen}
        logger.info("Round " + str(round_id) + " results: " + "P1//" + str(model1_gen) + ", P2//" + str(model2_gen))
        

In [ ]:
# save game hist
import json
out_fname = "llama7b_VS_llama70b"
with open(out_fname,'w+') as outf:
    json.dump(sv_game_history,outf)